In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics
import sklearn.calibration
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
# plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
from config import *
from tqdm import tqdm
import pickle as pkl
import train_reg
from math import floor
from copy import deepcopy
from sklearn.metrics import confusion_matrix
import config
import models
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
import neural_networks
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F
import interpret
import matplotlib.pyplot as plt
from functools import partial
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import viz

# prepare data

In [28]:
dsets = ['vps4_snf7']
splits = ['train', 'test']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized']
dfs, feat_names = data.load_dfs_for_lstm(dsets=dsets,
                                         splits=splits,
                                         filter_hotspots=True,
                                         filter_short=True,
                                         lifetime_threshold=3,
                                         hotspots_threshold=25,
                                         meta=meta,
                                         normalize=False)


print('type', type(dfs))
# load model
p = 1
for k in dfs:
    print(*k, dfs[k].shape)
df = dfs[list(dfs.keys())[0]]
X = df[feat_names[:p]]
y = df['y_consec_thresh']

100%|██████████| 1/1 [00:07<00:00,  7.73s/it]

type <class 'dict'>
vps4_snf7 train (7420, 90)
vps4_snf7 test (1922, 90)


**summarize dsets**

In [27]:
ds = dfs

vals = {} # key is dataset, Partition
for k in sorted(ds, key=lambda kv: (kv[0], kv[1])):
    d = ds[k]
    d.short = d.lifetime <= 15
    # print(k, d.lifetime.min())
    y = d['y_consec_thresh']
    feats = ['Total clean tracks',
             'Difficult tracks', 'Difficult valid events', 
             'Short tracks', 'Short valid events',]
    vals[k] = [
        d.shape[0],
        (~d.short).sum(), y[~d.short].sum(),
        d.short.sum(), y[d.short].sum()
    ]
    vals[k] = [int(x) for x in vals[k]]
#     print(feats, vals)
df = pd.DataFrame.from_dict(vals).transpose().round(decimals=0)
df.columns = feats
df.index = df.index.set_names(['Dataset', 'Partition'])
print(df.to_latex(index=True, index_names=True))

\begin{tabular}{llrrrrr}
\toprule
          &       &  Total clean tracks &  Difficult tracks &  Difficult valid events &  Short tracks &  Short valid events \\
Dataset & Partition &                     &                   &                         &               &                     \\
\midrule
vps4\_snf7 & test &                2993 &               244 &                     103 &          2749 &                 356 \\
          & train &                9174 &              1006 &                     411 &          8168 &                1332 \\
\bottomrule
\end{tabular}

